In [ ]:
import os
import re
import json
import random

import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt

from tqdm import tqdm

In [ ]:
DATA_PTH = '.\\dataset\\data_v0'
VOX_PTH = '.\\dataset\\chair_voxel_data'
RESOLUTION = 128

In [ ]:
def sorted_alphanumeric(data):
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ] 
    return sorted(data, key=alphanum_key)

In [ ]:
dir_pths = [os.path.join(DATA_PTH, p) for p in sorted_alphanumeric(os.listdir(DATA_PTH))]

In [ ]:
dir_number_with_chair = []

for dir_pth in tqdm(dir_pths):
    if 'meta.json' in os.listdir(dir_pth):
        with open(os.path.join(dir_pth, 'meta.json'), 'r') as f:
            j = json.load(f)
            if j['model_cat'] == 'Chair':
                dir_number = int(dir_pth.split('\\')[-1])
                dir_number_with_chair.append(dir_number)

dir_number_with_chair = np.array(dir_number_with_chair)

In [ ]:
dir_number_with_chair.shape

In [ ]:
np.save('dir_number_with_chair', dir_number_with_chair)

In [ ]:
dir_number_with_chair = np.load('.\\dataset\\dir_number_with_chair.npy')

obj_ct = 0

for dir_number in tqdm(dir_number_with_chair):

    dir_pth = os.path.join(DATA_PTH, str(dir_number))

    obj_dir_pth = os.path.join(dir_pth, 'objs')

    voxels = []

    for obj_name in sorted_alphanumeric(os.listdir(obj_dir_pth)):

        obj_pth = os.path.join(obj_dir_pth, obj_name)

        vox_pth = os.path.join(VOX_PTH, str(obj_ct))

        obj_ct += 1

        mesh = o3d.io.read_triangle_mesh(obj_pth)
        mesh.scale(RESOLUTION, center=np.array([0., 0., 0.]))

        voxel_grid = o3d.geometry.VoxelGrid.create_from_triangle_mesh(mesh, voxel_size=1)
        voxels.append(voxel_grid)

    o3d.visualization.draw_geometries([*voxels])

In [ ]:
dir_number_with_chair = np.load('.\\dataset\\dir_number_with_chair.npy')

times = 4
split = int(len(dir_number_with_chair) / times)
obj_ct = 0

for i in range(times):  
    all_voxels = []
      
    for dir_number in tqdm(dir_number_with_chair[i * split: (i+1) * split], desc="Processing Meshes"):
        dir_pth = os.path.join(DATA_PTH, str(dir_number))
        obj_dir_pth = os.path.join(dir_pth, 'objs')

        meshes = []

        for obj_name in sorted_alphanumeric(os.listdir(obj_dir_pth)):
            obj_pth = os.path.join(obj_dir_pth, obj_name)
            meshes.append(o3d.io.read_triangle_mesh(obj_pth))

        max_bound = np.max(meshes[0].get_max_bound())
        min_bound = np.min(meshes[0].get_min_bound())
        mesh_center = 0

        for mesh in meshes:
            if np.max(mesh.get_max_bound()) > max_bound:
                max_bound = np.max(mesh.get_max_bound())
            if np.min(mesh.get_min_bound()) < min_bound:
                min_bound = np.min(mesh.get_min_bound())
            mesh_center += mesh.get_center()

        mesh_center = mesh_center / len(meshes)
        scale = 1 / max_bound - min_bound
        
        all_voxels.append([])

        for mesh in meshes:
            mesh.translate(-mesh_center)  # centering to origin
            mesh.scale(RESOLUTION / scale, center=np.array([0., 0., 0.]))

            voxel_grid = o3d.geometry.VoxelGrid.create_from_triangle_mesh(mesh, voxel_size=1)

            voxels = []

            for voxel in voxel_grid.get_voxels():
                x, y, z = voxel_grid.get_voxel_center_coordinate(voxel.grid_index)

                grid_index_x = int((x + RESOLUTION / 2))
                grid_index_y = int((y + RESOLUTION / 2))
                grid_index_z = int((z + RESOLUTION / 2))

                grid_index_x = max(0, min(grid_index_x, RESOLUTION - 1))
                grid_index_y = max(0, min(grid_index_y, RESOLUTION - 1))
                grid_index_z = max(0, min(grid_index_z, RESOLUTION - 1))

                voxels.append([grid_index_x, grid_index_y, grid_index_z])

            all_voxels[-1].append(voxels)

    for combined in tqdm(all_voxels, desc="Saving Voxels"):
        for voxels in combined:
            vox_np_pth = os.path.join(VOX_PTH, str(obj_ct))
            obj_ct += 1
            np.save(vox_np_pth, voxels)

In [ ]:
dir_number_with_chair = np.load('.\\dataset\\dir_number_with_chair.npy')

objs_count = []

for dir_number in dir_number_with_chair:
    dir_pth = os.path.join(DATA_PTH, str(dir_number))
    obj_dir_pth = os.path.join(dir_pth, 'objs')
    objs_count.append(len(os.listdir(obj_dir_pth)))
    
np.save('.\\dataset\\each_chair_parts_count', np.array(objs_count, dtype=np.int32))

In [ ]:
def get_outlier_indices(arr):
    Q1 = np.percentile(arr, 25)
    Q3 = np.percentile(arr, 75)
    
    IQR = Q3 - Q1
    
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    outlier_indices = np.where((arr < lower_bound) | (arr > upper_bound))[0]
    
    return outlier_indices

each_chair_parts_count = np.load('.\\dataset\\each_chair_parts_count.npy')

outlier_indices = get_outlier_indices(each_chair_parts_count)

np.save('.\\dataset\\outlier_indices', outlier_indices)